# Model Evaluation: Quick Start Guide

This example demonstrates how to evaluate an existing entailment dataset using [Unitxt](https://www.unitxt.ai/). Unitxt is used to load the dataset, generate the input to the model, run inference and evaluate the results. This notebook is based on [this](https://www.unitxt.ai/en/latest/docs/examples.html#evaluate-an-existing-dataset-from-the-unitxt-catalog) example, and adapted to use Granite on Replicate. Get a Replicate API token [here](https://replicate.com/account/api-tokens).

## Load Dependencies

In [ ]:
%pip install replicate
%pip install git+https://github.com/ibm/unitxt
%pip install openai
%pip install litellm
%pip install diskcache
%pip install scikit-learn
%pip install git+https://github.com/ibm-granite-community/utils

from unitxt.api import evaluate, load_dataset
from unitxt.inference import CrossProviderInferenceEngine

from ibm_granite_community.notebook_utils import get_env_var

import nest_asyncio
nest_asyncio.apply()

## Load a dataset from the Unitxt catalog

In [ ]:
# Use the Unitxt APIs to load the wnli entailment dataset using the standard template in the catalog for relation task with 2-shot in-context learning.
# We set loader_limit to 20 to limit reduce inference time.
dataset = load_dataset(
    card="cards.wnli",
    template="templates.classification.multi_class.relation.default",
    format="formats.chat_api",
    num_demos=2,
    demos_pool_size=10,
    loader_limit=20,
    split="test",
)

## Instantiate the evaluation client

We are using a CrossProviderInferenceEngine inference engine that supply api access to providers such as:
watsonx, bam, openai, azure, aws and more.

In [ ]:
model = CrossProviderInferenceEngine(model="granite-3-8b-instruct", provider="replicate",credentials={'api_token': get_env_var('REPLICATE_API_TOKEN')})

## Generate predictions

In [ ]:
predictions = model(dataset)

## Evaluate the predictions to determine results

In [6]:
results = evaluate(predictions=predictions, data=dataset)

## Print the scores

In [ ]:
print("Global Results:")
print(results.global_scores.summary)

print("Instance Results:")
print(results.instance_scores.summary)